In [2]:
import pandas as pd
import numpy as np
import requests
from keys import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 3.1) Get account info
## Get infos from all accounts, including faulty entries
- First, get account info with the 'webget_user_id' endpoint of instagram-bulk-scraper-latest.p.rapidapi.com
- The resulting variable is
  'infos' with the columns ['name','acc_name','id', 'follower_cnt', 'following_cnt',
                             'post_cnt', 'bio', 'category']
- Note, that there are still some double names from the accounts_final variable, where
  it's not clear to this point which account is the official

In [4]:
# loading the result from the previous script 'GetAccounts.ipynb'
accounts_final = pd.read_csv('accounts_final.csv')
# delete the old index 
accounts_final = accounts_final.drop('Unnamed: 0', axis = 1)
# accounts_final

In [6]:
# choose some accounts for the purpose of explaining the code
accounts_final = accounts_final[4:10]
accounts_final

,name,acc_name,id
4,Alexander Engelhard,alexander_engelhard_csu,48111005815
5,Alexander Engelhard,alex.engelhardt,3679960301
6,Alexander Graf Lambsdorff,alexandergraflambsdorff,5760876068
7,Alexander Hoffmann,alexander.hoffmannmdb,5708286586
8,Alexander Hoffmann,lxhoffmann,1558234338
9,Ana-Maria Trăsnea,amtrasnea,26039470493


In [8]:
# prepare the API call
headers = {
    "x-rapidapi-key": rapid_API_insta_key,
	"x-rapidapi-host": "instagram-bulk-scraper-latest.p.rapidapi.com"
} 
# create an empty data frame
infos = pd.DataFrame(columns=['name','acc_name','id', 'follower_cnt', 'following_cnt',
                             'post_cnt', 'bio', 'category'])
# empty list for saving jsons
info_jsons =[]

In [10]:
# loop accounts
for j, row in accounts_final.iterrows():
    print(row['name'])
    # get the account name
    acc_name = row['acc_name']
    # empty dictionary for saving data temporarily
    info = {}
    # ural with specified account name
    url = f'https://instagram-bulk-scraper-latest.p.rapidapi.com/webget_user_id/{acc_name}'
    response = requests.get(url, headers=headers)

    # API call, get json
    info_json = response.json()
    # save jsons to avoid double API calls in case of errors
    info_jsons.append(info_json)

    # ensure successful API call 
    if (response.status_code == 200) & (info_json['status'] == 'ok'):

        # save data from the json file
        info = {
        # name
        'name': row['name'],
        # account name
        'acc_name':row['acc_name'],
        # account id
        'id': row['id'],
        # number of followers
        'follower_cnt': info_json['data']['followers'],
        # number of persons followed
        'following_cnt': info_json['data']['followings'],
        # number of posts
        'post_cnt': info_json['data']['post_count'],
        # Instagram bio
        'bio': info_json['data']['biography'],
        # category, e.g 'Politician' or 'Actor'
        'category': info_json['data']['category']
        }
    
        # Convert info dictionary to DataFrame and concatenate to final df infos
        info_df = pd.DataFrame([info])  # Create a DataFrame from the post dictionary
        infos = pd.concat([infos, info_df], ignore_index=True)  
        
    else:
        print('Request failed with code: ', response.status_code, '.', info_json.get('message'))

infos

Alexander Engelhard
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you
Alexander Engelhard
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you
Alexander Graf Lambsdorff
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you
Alexander Hoffmann
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you
Alexander Hoffmann
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you
Ana-Maria Trăsnea
Request failed with code:  500 . Action is not permitted. We sincerely apologize for the inconvenience caused to you


,name,acc_name,id,follower_cnt,following_cnt,post_cnt,bio,category


In [10]:
# save info
# infos.to_csv('Ergebnisse/info1.csv')

In [11]:
# load info
# infos = pd.read_csv('Ergebnisse/info1.csv')
# infos = infos.drop('Unnamed: 0', axis = 1)
# infos

## Filter account info due to bio and category
To get distinguish double account names from the offical accounts, 
- I create filters to identify accounts with political expressions:
      - bio includes buzz words like 'spd','abgeordnete','demokratie'
      - account name contains buzz words like 'mdb'
      - category can be 'Politician'
- I use filters to exclude accounts with non-political expressions:
      - categories like 'Artist','Health/beauty'
      - buzz words in the bio

In [13]:
# expressions to seach in bio for
expression = 'spd|cdu|csu|linke|afd|fdp|bsw|ndr|parlament|mdb|abgeordnete'
expression2 = 'vorsitzpolitik|bundes|wahlkreis|demokratie|minister|republik'
# categories to exclude
cat_list =['Artist', 'Health/beauty', 'Athlete', 'Digital creator','Real Estate Investment Firm',
       'Nutritionist', 'Fashion Model', 'Health & wellness website',
       'Actor', 'Medical & health', 'Photographer', 'Just for fun']
# expression to exclude
expression_ex = 'sexual|papaya'

# create mask with expressions to include
mask = (
    (infos['bio'].str.casefold().str.strip().str.contains(expression + expression2, regex=True)) | # expressions = True
    (infos['acc_name'].str.casefold().str.strip().str.contains('mdb|mdl')) |                       # mdb/mdl in acc_name = True
    (infos['category'] == 'Politician') |                                                          # category politician = True
    (infos['bio'] == '')                                                                           # include empty bios
)
# create mask with expressions to exclude
mask2 =(
    (infos['category'].isin(cat_list)) |                                                          # these categories = False
    (infos['bio'].str.casefold().str.strip().str.contains(expression_ex, regex=True, na=False))  # exclude these = False
)

# apply mask to infos
infos_final = infos.loc[(mask & ~mask2),:]


In [14]:
# save infos_final
# infos_final.to_csv('Ergebnisse/infos_final.csv')

# load infos final
#infos_final = pd.read_csv('Ergebnisse/infos_final.csv')
#infos_final = infos_final.drop('Unnamed: 0',axis = 1)

In [15]:
infos_final.head()

,name,acc_name,id,follower_cnt,following_cnt,post_cnt,bio,category


## Create infos_show variable for the web app
- add party information to the variable infos
- cleaning to have only info that is necessary for the user

In [17]:
# load election results to get party information
party = pd.read_csv('election2021.csv')
# only keep name and party info
party = party.loc[:,['name','party']]
party = party.astype({'party':'category'})
party.head()

,name,party
0,Achim Jürgen Post,SPD
1,Adis Ahmetovic,SPD
2,Agnes Monika Brugger,GRÜNE
3,Albert Robert Rupprecht,CSU
4,Albert Stegemann,CDU


In [18]:
# add party to infos_final and make pretty
infos_show = infos_final.merge(party, on='name')
# only info necessary for the user
infos_show = infos_show[['name', 'acc_name','party', 'follower_cnt', 'following_cnt', 'post_cnt','bio']]
# clearer names
infos_show = infos_show.rename(columns={'acc_name':'account'})
# infos_show.to_csv('infos_show.csv')
infos_show.head()

,name,account,party,follower_cnt,following_cnt,post_cnt,bio


# 3.2) Get posts
- First, get posts of every single account back to 2020-01-01 by using the webuser_posts-
  endpoint of the instagram-bulk-scraper-latest API of rapidapi.com.
- With one API call, 12 posts are pulled. If the account contains more posts, the
  has_next_page within the response is True and the 'end_cursor' is given to the next API
  request
- The resulting variable is
  'posts' with the columns ['name','acc_name','id','shortcode','date', 'media_type',
                              'like_cnt','comment_cnt', 'vid_view_count','comment','url']

In [20]:
# empty df for posts
posts = pd.DataFrame(columns=['name','acc_name','id','shortcode','date', 'media_type',
                              'like_cnt','comment_cnt', 'vid_view_count',
                              'comment','url'])
# empty list for saving jsons (account level)
post_jsons_accounts = []

# header for API call
headers = {
	'x-rapidapi-key': rapid_API_insta_key,
	'x-rapidapi-host': 'instagram-bulk-scraper-latest.p.rapidapi.com'
}

In [21]:
# specify to which date posts should be extracted
exit_time = pd.to_datetime('2020-01-01 00:00:00')

# loop the accounts with j 
for j, row in infos_final.iterrows():
    
    # Set flag for exit time to False (continues pulling posts)
    before_period = False
    # Set flag for account has more posts to True (continues pulling posts)
    has_next = True
    # Track the current status 
    print(row['name'])
    # Get the Instagram ID from accounts df
    account_id = row['id']
    
    # specifiy url
    url = f'https://instagram-bulk-scraper-latest.p.rapidapi.com/webuser_posts/{account_id}'
    # set querystring for first API call
    querystring = {'count':12,
                   'nocors':'false',
                   'end_cursor': None}
    
    # create empty list for saving jsons of one account
    post_jsons = []
    # track iteration of a pulled pages
    n = 0
    
    # loop one instagram account with an infinity loop 
    while True:     # needs to break at a timestamp
        
        if has_next:    # check if the account has a next page

            # API request
            response = requests.request('GET', url, headers=headers, params=querystring)

            # ensure a successful API call
            if response.status_code == 200:
                # Track pulled pages
                print('Page: ', n)
                
                post_json = response.json()
                # save all posts from one account
                post_jsons.append(post_json)    
                
                # Set flag if account has next page/more posts
                has_next = post_json['data']['page_info']['has_next_page'] 
                # identifier for the next posts
                querystring['end_cursor'] = post_json['data']['page_info']['end_cursor']
                
                post ={} # empty dictionary for the current 12 posts
                
                # loop all posts from one request (max 12 at a time)
                for item in post_json['data']['items']:
                    
                    # convert timestamp of post
                    time =  pd.to_datetime(item['node']['taken_at_timestamp'], unit = 's')
                    # Exit loop if time is before the given date
                    if time < exit_time:
                        before_period = True
                        break
                    
                    # see if caption is available
                    check = item['node']['edge_media_to_caption']['edges']
                    if check:
                        comment = check[0]['node']['text']
                    else:
                        comment = []
                    
                    # write info into temporal dictionary 
                    post =  {
                            # name of politician
                            'name': row['name'],
                            # account name
                            'acc_name':row['acc_name'],
                            # account id
                            'id':row['id'],
                            # shortcode, url identifier
                            'shortcode':item['node']['shortcode'],
                            # time
                            'date': time,
                            # media type
                            'media_type':item['node']['__typename'],
                            # count likes
                            'like_cnt': item['node']['edge_media_preview_like']['count'],
                            # count comments
                            'comment_cnt': item['node']['edge_media_to_comment']['count'],
                            # view count, or NaN if not available
                            'vid_view_count': item['node'].get('video_view_count', np.NaN),
                             # comment
                            'comment': comment,
                            # url of the media, see if video_url or get display url
                            'url': item['node'].get('video_url', item['node']['display_url'])
                            }
                
                    # Convert post dictionary to DataFrame and concatenate to posts
                    post_df = pd.DataFrame([post])  # Create a DataFrame from the post dictionary
                    posts = pd.concat([posts, post_df], ignore_index=True)  # Append to the posts DataFrame
            
            else:
                print(f'Request failed with status code: ', response.status_code)
                break
                
        n+=1 # count for pages
        
        # Exit infinity loop 
        if not has_next:   # break if has next = False
            break
        if before_period: # break if before_period = True
            break
            
    # save json with every account
    post_jsons_accounts.append(post_jsons)    

# save posts variable
# posts.to_csv('posts.csv')

posts

,name,acc_name,id,shortcode,date,media_type,like_cnt,comment_cnt,vid_view_count,comment,url


# 3.3) Postprocessing
Some postprocessing steps are needed to ensure the smooth operation of the web app
## Handling large dataframes
- As the database is large, some steps are needed:
- saving memory by changing the types to the minimum types necessary


In [23]:
# function for changing datatypes of posts dataframes 
def change_types(df):
    # change strings to category
    df['name'] = df['name'].astype('category')
    df['acc_name'] = df['acc_name'].astype('category')
    df['media_type'] = df['media_type'].astype('category')
    # save integers with the minimum bytes possible
    df['id'] = df['id'].astype('uint64')    
    df['like_cnt'] = df['like_cnt'].astype('int32')
    df['comment_cnt'] = df['comment_cnt'].astype('uint16')
    # fill NaNs with zeros
    df['vid_view_count'] = df['vid_view_count'].fillna(0)
    df['vid_view_count'] = df['vid_view_count'].astype('uint32')
    # change date to datetime 
    df['date'] = pd.to_datetime(df['date'])
    # show 
    df.info()
    return df

def change_types2(df):

    # fill NaNs with zeros
    df['vid_view_count'] = df['vid_view_count'].fillna(0)
    # change date to datetime 
    df['date'] = pd.to_datetime(df['date'])  

    # change datatypes
    df = df.astype({
                    # change strings to category
                    'media_type':'category',
                    # save integers with the minimum number of bytes possible
                    'id':'uint64',   
                    'like_cnt':'int32',
                    'comment_cnt':'uint16',
                    'vid_view_count':'uint32'
    })
                    
    # show 
    df.info()
    return df

In [24]:
# load posts and use function
posts = pd.read_csv('posts_all.csv')
posts = posts.drop('Unnamed: 0',axis=1)
posts = change_types2(posts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308762 entries, 0 to 308761
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   name            308762 non-null  object        
 1   acc_name        308762 non-null  object        
 2   id              308762 non-null  uint64        
 3   shortcode       308762 non-null  object        
 4   date            308762 non-null  datetime64[ns]
 5   media_type      308762 non-null  category      
 6   like_cnt        308762 non-null  int32         
 7   comment_cnt     308762 non-null  uint16        
 8   vid_view_count  308762 non-null  uint32        
 9   comment         308415 non-null  object        
 10  url             308751 non-null  object        
dtypes: category(1), datetime64[ns](1), int32(1), object(5), uint16(1), uint32(1), uint64(1)
memory usage: 19.7+ MB


## Create posts_show for the app
- add party information to the variable posts
- cleaning to have only info that is necessary for the user

In [26]:
# merge party information
posts_show = posts.copy().merge(party, on='name')
# create instagram url link using the account name and the shortcode
posts_show['webpage'] = 'https://www.instagram.com/' + posts['acc_name'] + '/p/' + posts['shortcode']
# sort df
posts_show = posts_show[['name','party','date','like_cnt','comment_cnt','vid_view_count','comment','webpage']]
# rename
posts_show = posts_show.rename(columns={'like_cnt':'likes',
                                       'comment_cnt':'comments',
                                       'vid_view_count':'video_views'})
# save as variable
# posts_show.to_csv('posts_show.csv')
posts_show.head()

,name,party,date,likes,comments,video_views,comment,webpage
0,Achim Jürgen Post,SPD,2024-11-02 07:55:20,211,4,0,"U3-Betreuungsquote, Armutsgefährdungsquote, Le...",https://www.instagram.com/achim_p/p/DB3IIFKtII5
1,Achim Jürgen Post,SPD,2024-11-01 12:29:25,25,0,0,Auf Einladung von Geschäftsführer Rudi Mantler...,https://www.instagram.com/achim_p/p/DB1Csn8qXGf
2,Achim Jürgen Post,SPD,2024-10-31 19:23:40,110,0,0,Leistungsträger:innen kommen nicht nur im Nade...,https://www.instagram.com/achim_p/p/DBzNTzhtLm_
3,Achim Jürgen Post,SPD,2024-10-31 11:19:00,17,0,0,Vom 18. bis zum 20.12. finden unter der Schirm...,https://www.instagram.com/achim_p/p/DByV17MKpg6
4,Achim Jürgen Post,SPD,2024-10-29 19:36:27,38,2,0,Nach über 70 Jahren ist das Kraftwerk Heyden v...,https://www.instagram.com/achim_p/p/DBuFLwTN62E


In [27]:
posts_show.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308762 entries, 0 to 308761
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   name         308762 non-null  object        
 1   party        308762 non-null  category      
 2   date         308762 non-null  datetime64[ns]
 3   likes        308762 non-null  int32         
 4   comments     308762 non-null  uint16        
 5   video_views  308762 non-null  uint32        
 6   comment      308415 non-null  object        
 7   webpage      308762 non-null  object        
dtypes: category(1), datetime64[ns](1), int32(1), object(3), uint16(1), uint32(1)
memory usage: 12.7+ MB


## Save posts in smaller chunks
- To handle the large dataframe within the webapp, the posts variable is split
  into smaller chunks
- when loading the variable later, this helps improving performance

In [29]:
posts_show.head()

,name,party,date,likes,comments,video_views,comment,webpage
0,Achim Jürgen Post,SPD,2024-11-02 07:55:20,211,4,0,"U3-Betreuungsquote, Armutsgefährdungsquote, Le...",https://www.instagram.com/achim_p/p/DB3IIFKtII5
1,Achim Jürgen Post,SPD,2024-11-01 12:29:25,25,0,0,Auf Einladung von Geschäftsführer Rudi Mantler...,https://www.instagram.com/achim_p/p/DB1Csn8qXGf
2,Achim Jürgen Post,SPD,2024-10-31 19:23:40,110,0,0,Leistungsträger:innen kommen nicht nur im Nade...,https://www.instagram.com/achim_p/p/DBzNTzhtLm_
3,Achim Jürgen Post,SPD,2024-10-31 11:19:00,17,0,0,Vom 18. bis zum 20.12. finden unter der Schirm...,https://www.instagram.com/achim_p/p/DByV17MKpg6
4,Achim Jürgen Post,SPD,2024-10-29 19:36:27,38,2,0,Nach über 70 Jahren ist das Kraftwerk Heyden v...,https://www.instagram.com/achim_p/p/DBuFLwTN62E


In [30]:
# saving into csv. with smaller size
var = posts_show
# define the chunksize
chunksize = 30000
# empty list for appending filenames
filenames=[]
# defines a list with row boundaries 
fromto = list(range(0,len(var),chunksize)) + [len(var)]

for i in range(0,len(fromto)-1):
    # define path 
    path = f'post_chunks{i}.csv'
    # saving the filenames into a list
    filenames.append(path)
    # saves only rows within the defined boundaries
    #var.loc[fromto[i]:fromto[i+1]-1].to_csv(path)

# save list of chunk filenames
filenames_df = pd.DataFrame(filenames,columns=['path'])
# filenames_df.to_csv('PostFilenames.csv', index=False)